In [10]:
import pandas as pd
data_df = pd.read_csv("train.csv", index_col='no', skipinitialspace=True)

In [11]:
X_df = data_df.iloc[:,:-1]
y_df = data_df.income

# preprocessing

In [12]:
from preprocessing import Preprocessing 
pp = Preprocessing()
X_df = pp.preprocess_hj(X_df)

In [13]:
def make_submission(model, name):
    sub_df = pd.read_csv("test.csv", index_col='no', skipinitialspace=True)
    sub_df = pp.preprocess_hj(sub_df)
    result = model.predict(sub_df)
    submission_df = pd.read_csv('sample_submission.csv')
    submission_df = submission_df.drop(columns='income')
    new_submission_df = pd.concat([submission_df, pd.DataFrame(result, columns=['income'])], axis=1)
    new_submission_df.to_csv(f'submission_{name}.csv', index=False)

# Test Set slicing

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size= 0.21, random_state=20)

## Answer Code

In [15]:
answer_df = pd.read_csv("../dataleakage/answer.csv", index_col='no', skipinitialspace=True)
answ_X_df = answer_df.iloc[:,:-1]
answ_y_df = answer_df.income
answ_X_df = pp.preprocess_hj(answ_X_df)

In [16]:
X_df

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_0,workclass_1,workclass_2,workclass_4,workclass_5,...,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,native-country_others
no,,,,,,,,,,,,,,,,,,,,,
1,25,7,0,0,40,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
2,39,10,0,1721,55,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
3,35,13,0,0,50,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
4,64,11,0,0,40,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
5,24,10,0,0,70,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29301,20,9,0,0,40,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,1
29302,21,9,0,0,40,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,1
29303,18,6,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1


# Xgboost

In [17]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

xgb_clf = xgb.XGBClassifier(
    # enable_categorical=True,
    gamma=0.25,
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    tree_method='auto',
    eval_metric='auc',
    n_jobs=-1,
    min_child_weight=1.1)

xgb_clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (answ_X_df, answ_y_df)], verbose=False)
xgb_roc_score = roc_auc_score(answ_y_df, xgb_clf.predict_proba(answ_X_df)[:,1], average='micro')
print("ROC score ", xgb_roc_score)

ROC score  0.8965839578760357


In [18]:
from sklearn.model_selection import cross_val_score
xgb_model_result = cross_val_score(xgb_clf, X_test, y_test, cv=10, n_jobs=-1, error_score='raise')

print('교차검증 결과:', xgb_model_result)
print('교차검증 결과 평균값', xgb_model_result.mean())
print('전체검증 결과:', xgb_clf.score(X_test,y_test))

교차검증 결과: [0.86850649 0.84577922 0.85227273 0.87175325 0.88474026 0.87479675
 0.8699187  0.87804878 0.87154472 0.89593496]
교차검증 결과 평균값 0.8713295850490972
전체검증 결과: 0.8770105605199026


In [19]:
xgb_model_result = cross_val_score(xgb_clf, answ_X_df, answ_y_df, cv=10, n_jobs=-1, error_score='raise')
print('실제 교차검증 결과:', xgb_model_result)
print('실제 교차검증 결과 평균값', xgb_model_result.mean())
print('실제 전체검증 결과:', xgb_clf.score(answ_X_df,answ_y_df))

실제 교차검증 결과: [0.84800409 0.85209826 0.84800409 0.84953941 0.84288639 0.85977482
 0.85005118 0.84741423 0.84895033 0.85407066]
실제 교차검증 결과 평균값 0.8500793467363283
실제 전체검증 결과: 0.8524850284076368


In [20]:
make_submission(xgb_clf,"xgb")

In [21]:
# # grid
# from sklearn.model_selection import GridSearchCV

# parameters = {'max_depth': [ 5, 6, 7],
#               'learning_rate': [0.04, 0.06, 0.08, 0.1, 0.12],
#               'iterations': [70, 80, 90, 100, 110]
#               }
# Grid_CBC = GridSearchCV(estimator=xgb_clf, param_grid=parameters, cv=5, n_jobs=-1, verbose=True)
# Grid_CBC.fit(X_train, y_train, verbose=True)

In [22]:
# print('final params', Grid_CBC.best_params_)   # 최적의 파라미터 값 출력
# print('best score', Grid_CBC.best_score_)      # 최고의 점수

# AdaBoost

In [23]:
from sklearn.ensemble import AdaBoostClassifier

bagging_adb = AdaBoostClassifier(n_estimators=300, learning_rate=0.05, random_state=42)
bagging_adb.fit(X_train, y_train)
adb_roc_score = roc_auc_score(y_test, bagging_adb.predict_proba(X_test)[:,1], average='micro')

print("훈련 세트 정확도 : {:.3f}".format(bagging_adb.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(bagging_adb.score(answ_X_df, answ_y_df)))
print("ROC score ", adb_roc_score)

훈련 세트 정확도 : 0.850
테스트 세트 정확도 : 0.842
ROC score  0.9178929888635119


In [24]:
adb_model_result = cross_val_score(bagging_adb, answ_X_df, answ_y_df, cv=10, n_jobs=-1, error_score='raise')
print('실제 교차검증 결과:', adb_model_result)
print('실제 교차검증 결과 평균값', adb_model_result.mean())
print('실제 전체검증 결과:', bagging_adb.score(answ_X_df,answ_y_df))

실제 교차검증 결과: [0.84237462 0.84646878 0.83418628 0.84288639 0.83930399 0.84851586
 0.83520983 0.84024578 0.83666155 0.8453661 ]
실제 교차검증 결과 평균값 0.84112191778022
실제 전체검증 결과: 0.8417873777959769


# GBM 

In [25]:
from sklearn.ensemble import GradientBoostingClassifier 
gbm_clf = GradientBoostingClassifier(max_depth=4,max_features='log2', n_estimators=150, learning_rate=0.15)
gbm_clf.fit(X_train, y_train)

gbm_roc_score = roc_auc_score(y_test, bagging_adb.predict_proba(X_test)[:,1], average='micro')
print("ROC score ", gbm_roc_score)

ROC score  0.9178929888635119


In [26]:
print("훈련 세트 정확도 : {:.3f}".format(gbm_clf.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(gbm_clf.score(answ_X_df, answ_y_df)))

훈련 세트 정확도 : 0.872
테스트 세트 정확도 : 0.848


In [27]:
gbm_model_result = cross_val_score(gbm_clf, answ_X_df, answ_y_df, cv=10, n_jobs=-1, error_score='raise')
print('실제 교차검증 결과:', gbm_model_result)
print('실제 교차검증 결과 평균값', gbm_model_result.mean())
print('실제 전체검증 결과:', gbm_clf.score(answ_X_df,answ_y_df))

실제 교차검증 결과: [0.85005118 0.8531218  0.84135107 0.84390993 0.84800409 0.85926305
 0.84493347 0.84280594 0.84434204 0.8515105 ]
실제 교차검증 결과 평균값 0.847929306984347
실제 전체검증 결과: 0.8476224599477914


# lightGBM

In [28]:
import lightgbm as lgb

# 트레이닝 데이터를 LightGBM에 맞는 데이터 세트 포맷으로 변환
# LightGBM 트레이닝에 있어서 필수적
# 모델의 정확도는 설정된 파라미터 값에 전적으로 달려 있음
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.1
# rf (랜덤포레스트), dart, goss
params['boosting_type'] = 'gbdt'

# 이진분류 : 초기값은 회귀 분석 (regression (회귀), multiclass (다중 분류))
params['objective'] = 'binary'

# mae, mse, multi_logloss
params['metric'] = 'binary_logloss'

# 특성 비율
params['sub_feature'] = 0.5
# 2^max_depth보다 작아야 한다
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 5

# 훈련
clf = lgb.train(params, d_train, 700)

[LightGBM] [Info] Number of positive: 5573, number of negative: 17577
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000525 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 395
[LightGBM] [Info] Number of data points in the train set: 23150, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240734 -> initscore=-1.148658
[LightGBM] [Info] Start training from score -1.148658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [29]:
# 예측
y_pred = clf.predict(X_test)

# 이진 값으로 변환
for i in range(0, y_pred.shape[0]):
    if y_pred[i]>=.5:       
       y_pred[i]=1
    else:  
       y_pred[i]=0

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.8761982128350935

In [30]:
# 예측
y_pred = clf.predict(answ_X_df)

# 이진 값으로 변환
for i in range(0, y_pred.shape[0]):
    if y_pred[i]>=.5:       
       y_pred[i]=1
    else:  
       y_pred[i]=0

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, answ_y_df)
accuracy

0.8506423708860111

# catBoost

In [31]:
from teddynote import models

In [32]:
from catboost import CatBoostClassifier

clf_catboost = CatBoostClassifier(iterations=200, max_depth=6, learning_rate=0.08)
clf_catboost.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=True)

0:	learn: 0.6132930	test: 0.6117327	best: 0.6117327 (0)	total: 154ms	remaining: 30.7s
1:	learn: 0.5529571	test: 0.5498904	best: 0.5498904 (1)	total: 163ms	remaining: 16.2s
2:	learn: 0.5086746	test: 0.5050168	best: 0.5050168 (2)	total: 174ms	remaining: 11.4s
3:	learn: 0.4740389	test: 0.4697896	best: 0.4697896 (3)	total: 182ms	remaining: 8.92s
4:	learn: 0.4440721	test: 0.4389950	best: 0.4389950 (4)	total: 189ms	remaining: 7.36s
5:	learn: 0.4188939	test: 0.4128732	best: 0.4128732 (5)	total: 197ms	remaining: 6.37s
6:	learn: 0.4013252	test: 0.3948980	best: 0.3948980 (6)	total: 204ms	remaining: 5.62s
7:	learn: 0.3886132	test: 0.3820912	best: 0.3820912 (7)	total: 213ms	remaining: 5.1s
8:	learn: 0.3799418	test: 0.3733994	best: 0.3733994 (8)	total: 221ms	remaining: 4.69s
9:	learn: 0.3698459	test: 0.3628314	best: 0.3628314 (9)	total: 228ms	remaining: 4.33s
10:	learn: 0.3622677	test: 0.3549321	best: 0.3549321 (10)	total: 236ms	remaining: 4.05s
11:	learn: 0.3554400	test: 0.3477775	best: 0.3477775 

In [33]:
clf_catboost.score(X_test,y_test)
# cat_roc_score = roc_auc_score(y_test, clf_catboost.predict_proba(X_test)[:,1], average='micro')
# print("ROC score ", cat_roc_score)

0.8778229082047116

In [34]:
gbm_model_result = cross_val_score(clf_catboost, X_test, y_test, cv=10, n_jobs=-1, error_score='raise')
print('교차검증 결과:', gbm_model_result)
print('교차검증 결과 평균값', gbm_model_result.mean())

교차검증 결과: [0.87337662 0.8474026  0.85064935 0.85876623 0.87662338 0.8699187
 0.85691057 0.87317073 0.8699187  0.89268293]
교차검증 결과 평균값 0.8669419807834442


In [35]:
gbm_model_result = cross_val_score(clf_catboost, answ_X_df, answ_y_df, cv=10, n_jobs=-1, error_score='raise')
print('실제 교차검증 결과:', gbm_model_result)
print('실제 교차검증 결과 평균값', gbm_model_result.mean())
print('실제 전체검증 결과:', clf_catboost.score(answ_X_df,answ_y_df))

cat_roc_score = roc_auc_score(answ_y_df, clf_catboost.predict_proba(answ_X_df)[:,1], average='micro')
print("ROC score ", cat_roc_score)

실제 교차검증 결과: [0.85363357 0.85465711 0.8444217  0.84902764 0.84953941 0.85823951
 0.85209826 0.84587814 0.84946237 0.85304659]
실제 교차검증 결과 평균값 0.8510004292270612
실제 전체검증 결과: 0.8514101448533552
ROC score  0.8960614073033712


In [36]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'max_depth': [6, 7, 8, 9],
#               'learning_rate': [0.04, 0.06, 0.08, 0.1],
#               'iterations': [100, 150, 200, 250, 300]
#               }
# Grid_CBC = GridSearchCV(estimator=clf_catboost, param_grid=parameters, cv=2, n_jobs=-1, verbose=True)
# Grid_CBC.fit(X_train, y_train, eval_set=(answ_X_df, answ_y_df), verbose=True)



In [37]:
# print('final params', Grid_CBC.best_params_)   # 최적의 파라미터 값 출력
# print('best score', Grid_CBC.best_score_)      # 최고의 점수

# voting

In [49]:
from sklearn.ensemble import VotingClassifier
# voting = VotingClassifier(estimators=[("XGB", xgb_clf), ("ADA", bagging_adb), ("GBM", gbm_clf), ("cat",clf_catboost)], voting ='hard', verbose=True, n_jobs=-1)
voting = VotingClassifier(estimators=[("XGB", xgb_clf), ("cat",clf_catboost)], voting ='soft', verbose=True, n_jobs=-1)
voting.fit(X_train, y_train)
voting.score(X_test, y_test)

0.8789601949634444

In [53]:
vote_result = cross_val_score(voting, X_test, y_test, cv=5, n_jobs=-1, error_score='raise')
print('교차검증 결과:', vote_result)
print('교차검증 결과 평균값', vote_result.mean())

교차검증 결과: [0.85702681 0.85540211 0.8773355  0.87246141 0.88870837]
교차검증 결과 평균값 0.870186839967506


In [54]:
vote_result = cross_val_score(voting, answ_X_df, answ_y_df, cv=10, n_jobs=-1, error_score='raise')
print('실제 교차검증 결과:', vote_result)
print('실제 교차검증 결과 평균값', vote_result.mean())
print('실제 전체검증 결과:', voting.score(answ_X_df,answ_y_df))

cat_roc_score = roc_auc_score(answ_y_df, voting.predict_proba(answ_X_df)[:,1], average='micro')
print("ROC score ", cat_roc_score)

실제 교차검증 결과: [0.85158649 0.85465711 0.84493347 0.85158649 0.8444217  0.85619243
 0.85261003 0.84639017 0.8484383  0.85407066]
실제 교차검증 결과 평균값 0.850488684704685
실제 전체검증 결과: 0.8524338434764805
ROC score  0.8970187983990764


In [52]:
make_submission(voting, "vote")

# 